In [1]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache, SQLiteCache
from langchain_core.output_parsers import StrOutputParser
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.documents import Document
from langchain_community.document_loaders import PyMuPDFLoader

from ragas import EvaluationDataset, RunConfig, evaluate
from ragas.metrics import (
    LLMContextRecall, Faithfulness, LLMContextPrecisionWithReference, AnswerRelevancy
)
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

import pandas as pd

import os
import re

from dotenv import load_dotenv
load_dotenv()

def get_file_names(folder_path, format=".pdf"):
    """
    주어진 폴더 내에 있는 PDF 파일들의 이름을 리스트로 반환합니다.
    """
    import os
    
    try:
        all_files = os.listdir(folder_path)
        pdf_files = [file.replace(format,"") for file in all_files if file.lower().endswith(format)]
        
        return pdf_files
    except FileNotFoundError:
        print(f"Error: 폴더 '{folder_path}'를 찾을 수 없습니다.")
        return []
    except Exception as e:
        print(f"Error: {e}")
        return []
    

In [2]:
from loader import load_process_split_doc, load_process_split_doc_law

# tax_etc 파일들 정의의
doc_options = [
    {
        "filename":"연말정산_신고안내", 
        "page_ranges": [(17, 426)],
        "table_pages_range": "17-425",
        "replace_string": [
            {"target_str": r"01. 2024년 귀속 연말정산 개정세법 요약\n\d+|"},
            {"target_str": r"01. 2024년 귀속 연말정산 개정세법 요약\n\d+|"  },
            {"target_str": r"II. 2024년 귀속 연말정산 주요 일정\n\d+|"},
            {"target_str": r"III\. 원천징수의무자의 연말정산 중점 확인사항\n\d+|"},
            {"target_str": r"원천징수의무자를 위한 \n2024년 연말정산 신고안내\n\d+|"},
            {"target_str": r"Ⅰ\. 근로소득\n\d+|"},
            {"target_str": r"II\. 근로소득 원천징수 및 연말정산\n\d+|"},
            {"target_str": r"III\. 근로소득공제, 인적공제, 연금보험료공제\n\d+|"},
            {"target_str": r"IV\. 특별소득공제(소법 §52)\n\d+|"},
            {"target_str": r"V\. 그 밖의 소득공제(조특법)\n\d+|"},
            {"target_str": r"VI\. 세액감면(공제) 및 농어촌특별세\n\d+|"},
            {"target_str": r"I\. 2024년 귀속 연말정산 종합사례\n\d+|"},
            {"target_str": r"II\. 근로소득 원천징수영수증(지급명세서) 작성요령\n\d+|"},
            {"target_str": r"IV\. 수정 원천징수이행상황신고서 작성사례(과다공제)\n\d+|"},
            {"target_str": r"VI\. 홈택스를 이용한 연말정산 신고(근로소득 지급명세서 제출)\n\d+|"},
            {"target_str": r"I\. 사업소득 연말정산\n\d+|"},
            {"target_str": r"II\. 연금소득 연말정산\n\d+|"},
            {"target_str": r"I\. 종교인소득이란?\n\d+|"},
            {"target_str": r"IV\. 종교인소득(기타소득)에 대한 연말정산\n\d+|"},
            {"target_str": r"부록1\. 연말정산 관련 서비스\n\d+|"},
            {"target_str": r"부록2\. 연말정산간소화 서비스\n\d+|"},
            {"target_str": r"부록5\. 연말정산 주요 용어 설명\n\d+|"},
            {"target_str": r"부록6\. 소득·세액공제신고서 첨부서류\n\d+|"},
            {"target_str": r"\bNaN\b"},
            {"target_str": r"([\uAC00-\uD7A3])\n+([\uAC00-\uD7A3])","replace_str": r"\1\2"},
            {"target_str": r"\s+", "replace_str": r" "},
        ]
    },
    {
        "filename":"연말정산_주택자금·월세액_공제의이해", 
        "page_ranges": [(9, 12), (15, 24), (27, 28), (31, 39), (43, 70)],
        "table_pages_range": "9-12,15-24,27-28,31-39,43-70",
        "replace_string": [
            {"target_str": (r"연말정산 주택자금･월세액 공제의 이해\n\d+|" r"\bNaN\b"),  },
            {"target_str": r"([\uAC00-\uD7A3])\n+([\uAC00-\uD7A3])", "replace_str":  r"\1\2" },
            {"target_str": r"\s+", "replace_str": " " },
        ]
    },
    {
        "filename":"2024_핵심_개정세법", 
        "page_ranges": [(19, 44), (47, 71), (75, 161)],
        "table_pages_range": "19-44,47-71,75-161",
        "replace_string": [
            {"target_str": (r"2\n0\n2\n5\n\s*달\n라\n지\n는\n\s*세\n금\n제\n도|"  
                r"\n2\n0\n2\n4\n\s*세\n목\n별\n\s*핵\n심\n\s*개\n정\n세\n법|"
                r"\n2\n0\n2\n4\n\s*개\n정\n세\n법\n\s*종\n전\n-\n개\n정\n사\n항\n\s*비\n교\n|"
                r"\s*3\s*❚국민･기업\s*납세자용\s*|"
                r"\s*2\s*0\s*2\s*4\s|"
                r"\s한국세무사회\s|" 
                r"\n7\n❚국민･기업 납세자용|"
                r"\n71\n❚상세본|"),  
            },
            {"target_str": r"([\uAC00-\uD7A3])\n+([\uAC00-\uD7A3])", "replace_str":  r"\1\2" },
            {"target_str": r"\s+", "replace_str": " " },
        ]
    },
    {
        "filename":"주요_공제_항목별_계산사례", 
        "page_ranges": [(1,1), (4, 21)],
        "table_pages_range": "1,4-21",
        "replace_string": [
            {"target_str": r"\bNaN\b"},
            {"target_str": r"\s+", "replace_str": " " },
        ]
    }
    
]

# vector store 에 넣을 데이터 document 모으기 -> all_documents
all_documents = []
# load - 세법 
law_files = get_file_names("data/tax_law")
for file in law_files:
    all_documents.extend(load_process_split_doc_law(file))
# load - 참고자료
for doc in doc_options:
    all_documents.extend(load_process_split_doc(**doc))
    

c:\Users\Playdata\AppData\Local\miniconda3\envs\langchain2\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 15 of document data/tax_etc/연말정산_신고안내.pdf
  warnings.warn(
c:\Users\Playdata\AppData\Local\miniconda3\envs\langchain2\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 37 of document data/tax_etc/연말정산_신고안내.pdf
  warnings.warn(
c:\Users\Playdata\AppData\Local\miniconda3\envs\langchain2\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 223 of document data/tax_etc/연말정산_신고안내.pdf
  warnings.warn(
c:\Users\Playdata\AppData\Local\miniconda3\envs\langchain2\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 1 of document data/tax_etc/연말정산_주택자금·월세액_공제의이해.pdf
  warnings.warn(
c:\Users\Playdata\AppData\Local\miniconda3

In [3]:
# len(all_documents)

In [4]:
# 📌 vector store 생성
COLLECTION_NAME = "tax_law"
PERSIST_DIRECTORY = "tax"

def set_vector_store(documents):
    embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")

    return Chroma.from_documents(
        documents=documents,
        embedding=embedding_model,
        collection_name=COLLECTION_NAME,
        persist_directory=PERSIST_DIRECTORY
    )
    
vector_store = set_vector_store(all_documents)

In [5]:
COLLECTION_NAME = "tax_law"
PERSIST_DIRECTORY = "tax"

# 📌 vector store 불러오기

embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")
vector_store = Chroma(
        embedding_function=embedding_model,
        collection_name=COLLECTION_NAME,
        persist_directory=PERSIST_DIRECTORY
)

In [6]:
# vector_store._collection.count()

In [20]:
# Retreiver 생성
retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 3, "fetch_k": 5}
)

In [21]:
# Prompt Template 생성
messages = [
    ("ai", """
    당신은 대한민국 세법에 대해 전문적으로 학습된 AI 도우미입니다. 사용자의 질문에 대해 저장된 세법 조항 데이터와 관련 정보를 기반으로 정확하고 신뢰성 있는 답변을 제공하세요. 

    **역할 및 기본 규칙**:
    - 당신의 주요 역할은 세법 정보를 사용자 친화적으로 전달하는 것입니다.
    - 데이터에 기반한 정보를 제공하며, 데이터에 없는 내용은 임의로 추측하지 않습니다.
    - 불확실한 경우, "잘 모르겠습니다."라고 명확히 답변하고, 사용자가 질문을 더 구체화하도록 유도합니다.

    **질문 처리 절차**:
    1. **질문의 핵심 내용 추출**:
        - 질문을 형태소 단위로 분석하여 조사를 무시하고 핵심 키워드만 추출합니다. 
        - 질문의 형태가 다르더라도 문맥의 의도가 같으면 동일한 질문으로 간주합니다.
        - 예를 들어, "개별소비세법 1조 알려줘" 와 "개별소비세법 1조는 뭐야" 와 "개별소비세법 1조의 내용은?"는 동일한 질문으로 간주합니다.
        - 예를 들어, "소득세는 무엇인가요?"와 "소득세가 무엇인가요?"는 동일한 질문으로 간주합니다.
    2. **관련 세법 조항 검색**:
        - 질문의 핵심 키워드와 가장 관련 있는 세법 조항이나 시행령을 우선적으로 찾습니다.
        - 필요한 경우, 질문과 연관된 추가 조항도 검토하여 답변의 완성도를 높입니다.
    3. **질문 유형 판단**:
        - **정의 질문**: 특정 용어나 제도의 정의를 묻는 경우.
        - **절차 질문**: 특정 제도의 적용이나 신고 방법을 묻는 경우.
        - **사례 질문**: 구체적인 상황에 대한 세법 해석을 요청하는 경우.
    4. **답변 생성**:
        - 법률 조항에관한 질문이라면 그 조항에 관한 전체 내용을 가져온다.
        - 예를들어 '개별소비세법 1조의 내용'이라는 질문을 받으면 개별소비세법 1조의 조항을 전부 다 답변한다.
        - 질문 유형에 따라 관련 정보를 구조적으로 작성하며, 중요 세법 조문과 요약된 내용을 포함합니다.
        - 비전문가도 이해할 수 있도록 용어를 친절히 설명합니다.

    **답변 작성 가이드라인**:
    - **간결성**: 답변은 간단하고 명확하게 작성하되, 법 조항에 관한 질문일 경우 관련 법 조문의 전문을 명시합니다.
    - **구조화된 정보 제공**:
        - 세법 조항 번호, 세법 조항의 정의, 시행령, 관련 규정을 구체적으로 명시합니다.
        - 복잡한 개념은 예시를 들어 설명하거나, 단계적으로 안내합니다.
    - **신뢰성 강조**:
        - 답변이 법적 조언이 아니라 정보 제공 목적임을 명확히 알립니다.
        - "이 답변은 세법 관련 정보를 바탕으로 작성되었으며, 구체적인 상황에 따라 전문가의 추가 조언이 필요할 수 있습니다."를 추가합니다.
    - **정확성**:
        - 법령 및 법률에 관한질문은 추가적인 내용없이 한가지 content에 집중하여 답변한다.
        - 법조항에대한 질문은 시행령이나 시행규칙보단 해당법에서 가져오는것에 집중한다.
      
    **추가적인 사용자 지원**:
    - 답변 후 사용자에게 주제와 관련된 후속 질문 두 가지를 제안합니다.
    - 후속 질문은 사용자가 더 깊이 탐구할 수 있도록 설계하며, 각 질문 앞뒤에 한 줄씩 띄어쓰기를 합니다.

    **예외 상황 처리**:
    - 사용자가 질문을 모호하게 작성한 경우:
        - "질문이 명확하지 않습니다. 구체적으로 어떤 부분을 알고 싶으신지 말씀해 주시겠어요?"와 같은 문구로 추가 정보를 요청합니다.
    - 질문이 세법과 직접 관련이 없는 경우:
        - "이 질문은 제가 학습한 대한민국 세법 범위를 벗어납니다."라고 알리고, 세법과 관련된 새로운 질문을 유도합니다.

    **추가 지침**:
    - 개행문자 두 개 이상은 절대 사용하지 마세요.
    - 질문 및 답변에서 사용된 세법 조문은 최신 데이터에 기반해야 합니다.
    - 질문이 복합적인 경우, 각 하위 질문에 대해 별도로 답변하거나, 사용자에게 우선순위를 확인합니다.

    **예시 답변 템플릿**:
    - "질문에 대한 답변: ..."
    - "관련 세법 조항: ..."
    - "추가 설명: ..."

    {context}
    """),
    ("human", "{question}"),
]
prompt_template = ChatPromptTemplate(messages)
# 모델
model = ChatOpenAI(model="gpt-4o")

# output parser
parser = StrOutputParser()

# Chain 구성 retriever(관련문서 조회) -> prompt_template(prompt 생성) -> model(정답) -> output parser
chain = {"context":retriever, "question": RunnablePassthrough()} | prompt_template | model | parser

In [15]:
print(chain.invoke("법인세법 제1조 에 대해 알려줘"))
retriever.invoke("법인세법 제1조 에 대해 알려줘")

법인세법 제1조는 법인세법의 목적을 규정하고 있는 조항입니다. 일반적으로 법인세법 제1조는 법인세의 부과와 징수에 관한 사항을 규정하여 국가 재정에 기여하고, 법인 간의 과세 형평성을 도모하는 것을 목적으로 합니다. 법인세법이 어떻게 구성되어 있는지에 대한 전반적인 방향성을 제시하는 조항입니다.

**법인세법 제1조:**

제1조(목적) 이 법은 법인세의 부과와 징수에 관하여 필요한 사항을 규정함으로써 국가 재정에 기여함을 목적으로 한다.

이 조항은 법인세법이 국가 재정에 어떻게 기여하는지를 설명하고, 법인세 부과와 관련된 일련의 규정을 마련하는 데 그 목적이 있다는 것을 명확히 하고 있습니다. 

이 답변은 세법 관련 정보를 바탕으로 작성되었으며, 구체적인 상황에 따라 전문가의 추가 조언이 필요할 수 있습니다.

추가로 궁금하신 점이 있다면 아래와 같은 질문을 고려해 볼 수 있습니다:

- 법인세의 부과 기준은 어떻게 되나요?
  
- 법인세법의 주요 변경 사항은 무엇인가요?


[Document(metadata={'description': '상속세 및 증여세법 시행령 부칙-제31조에 관한 문서입니다.', 'keyword': '상속세 및 증여세법 시행령', 'source': '상속세_및_증여세법_시행령.pdf'}, page_content='부칙-제31조  삭제'),
 Document(metadata={'description': '상속세 및 증여세법 시행령 부칙-제31조에 관한 문서입니다.', 'keyword': '상속세 및 증여세법 시행령', 'source': '상속세_및_증여세법_시행령.pdf'}, page_content='부칙-제31조  삭제'),
 Document(metadata={'description': '상속세 및 증여세법 제81조에 관한 문서입니다.', 'keyword': '상속세 및 증여세법', 'source': '상속세_및_증여세법.pdf'}, page_content='제81조  삭제')]

In [28]:
print(chain.invoke("연봉 4000에 회사를 다니는 근로자인데 개인사업도 하고있어. 개인사업으로는 올해 3000만원의 소득이 이루어졌는데 연말정산을 어떻게 해야할까?"))

질문하신 경우는 근로소득과 사업소득을 동시에 가지고 있는 경우로, 연말정산과 종합소득세 신고가 필요합니다. 아래에 간단한 절차를 설명드리겠습니다.

### 1. 근로소득 연말정산
- **근로소득에 대한 연말정산**은 회사에서 매년 말에 자동으로 진행됩니다. 이 과정에서 본인의 연봉 4,000만 원에 대해 각종 소득공제 및 세액공제가 적용됩니다.

### 2. 사업소득 신고
- **사업소득에 대한 신고**는 종합소득세 신고를 통해 이루어집니다. 사업소득 3,000만 원에 대한 종합소득세 신고는 다음 해 5월에 진행됩니다.
- 이때, 사업소득에 대한 소득금액과 각종 경비 및 공제를 적용하여 과세표준을 계산하고, 이에 따른 세액을 납부하게 됩니다.

### 3. 종합소득세 신고
- **종합소득세 신고** 시에는 근로소득과 사업소득을 합산하여 신고합니다. 
- 근로소득에 대한 원천징수 영수증을 회사로부터 수령하고, 사업소득에 대한 매출·매입 내역 및 경비를 정리하여 종합소득세 신고를 준비합니다.
- 국세청 홈택스 시스템을 통해 전자신고를 하거나, 세무사에게 의뢰하여 신고할 수 있습니다.

### 추가 설명
- **소득공제 및 세액공제**: 근로소득과 사업소득을 합산한 총 소득에 대해 종합소득공제, 세액공제를 적용받을 수 있습니다. 이에 따라 최종 납부할 세액이 결정됩니다.
- **세액정산**: 근로소득에 대해 이미 원천징수된 세액을 사업소득에 대한 종합소득세 신고 시에 정산하여 추가 납부하거나 환급받을 수 있습니다.

이 과정에서 추가적인 소득공제나 세액공제를 받을 수 있는 항목이 있는지 확인하시고, 필요하다면 세무 전문가의 도움을 받는 것도 좋은 방법입니다.

**이 답변은 세법 관련 정보를 바탕으로 작성되었으며, 구체적인 상황에 따라 전문가의 추가 조언이 필요할 수 있습니다.**

혹시 더 궁금하신 점이 있으신가요? 예를 들어:

- 연말정산에서 받을 수 있는 소득공제 항목은 무엇인가요?
- 종합소득세 신고 방법에 대해 자세히 알고 싶어요.


In [10]:
print(chain.invoke("현재 농사를 법인회사 형태로 소득을 벌어들이고있는데 세금혜택이있을까?"))

농업 법인으로 운영하시는 경우 몇 가지 세금 혜택을 받을 수 있습니다. 다만, 구체적인 혜택은 법인의 형태, 규모, 농업의 종류 등에 따라 다를 수 있습니다. 일반적으로 농업 법인이 받을 수 있는 세금 혜택에는 다음과 같은 것들이 있습니다:

1. **법인세 감면**: 농업 법인은 일정 조건을 만족할 경우 법인세 감면 혜택을 받을 수 있습니다. 예를 들어, 농업회사법인으로 등록된 경우 소득세 또는 법인세 감면 혜택을 받을 수 있습니다.

2. **부가가치세 면제**: 농산물의 생산 및 판매와 관련된 부가가치세가 면제될 수 있습니다. 농업 법인이 직접 생산한 농산물을 직접 판매하는 경우 부가가치세가 면제됩니다.

3. **농업소득세 감면**: 농업 법인이 농산물의 생산 및 유통을 주로 하는 경우 농업소득세 감면 혜택을 받을 수 있습니다.

4. **지방세 감면**: 농업 법인이 사용하는 농지 및 농업 관련 시설에 대한 재산세, 취득세 등의 지방세가 감면될 수 있습니다.

각 혜택은 일정한 조건을 만족해야 하며, 법 개정에 따라 변경될 수 있습니다. 따라서, 구체적인 혜택을 확인하시려면 세무 전문가와 상담하시거나 관련 법령을 확인하는 것이 좋습니다.

이 답변은 세법 관련 정보를 바탕으로 작성되었으며, 구체적인 상황에 따라 전문가의 추가 조언이 필요할 수 있습니다.

추가적으로 궁금하신 점이 있다면 다음과 같은 질문을 고려해 보실 수 있습니다:

1. 농업 법인 설립 시 필요한 절차는 무엇인가요?
   
2. 농업 법인이 받을 수 있는 각종 지원금이나 보조금은 어떤 것이 있나요?


## 평가

In [33]:
from ragas import EvaluationDataset, RunConfig, evaluate
from datasets import Dataset
from ragas.metrics import (LLMContextRecall, Faithfulness, LLMContextPrecisionWithReference, AnswerRelevancy)
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall

# LangChain 모델 래핑
langchain_model = LangchainLLMWrapper(model)

# 테스트 데이터 준비
test_data = [
    # {
    #     "question": "주세법의 목적은 무엇인가요?",
    #     "answer": chain.invoke("주세법의 목적은 무엇인가요?"),
    #     "contexts": [doc.page_content for doc in retriever.get_relevant_documents("주세법의 목적은 무엇인가요?")],
    #     "ground_truths": ["주세의 과세 요건 및 절차를 규정함으로써 주세를 공정하게 과세하고, 납세의무의 적정한 이행을확보하며, 재정수입의 원활한 조달에 이바지함을 목적으로 한다."],
    #     "reference": "\n".join([doc.page_content for doc in retriever.get_relevant_documents("주세법의 목적은 무엇인가요?")])
    # },
    # {
    #     "question": "인적공제에서 공제금액을 알려주세요.",
    #     "answer": chain.invoke("인적공제에서 공제금액을 알려주세요."),
    #     "contexts": [doc.page_content for doc in retriever.get_relevant_documents("인적공제에서 공제금액을 알려주세요.")],
    #     "ground_truths": ["기본공제대상자 1명당 150만원. 기본공제대상자가 다음에 해당되는 경우 기본공제 외에 추가로 공제, 경로우대자(70세 이상)인 경우 1명당 연 100만원, 장애인인 경우 1명당 연 200만원, 종합소득금액이 3천만원 이하인 거주자가 배우자가 없는 여성으로서 기본공제대상부양가족이 있는세대주이거나, 배우자가 있는 여성근로자인 경우 연 50만원, 배우자가 없는 근로자가 기본공제대상 직계비속 또는 입양자가 있는 경우 연 100만원(부녀자공제와 중복 배제：한부모 공제를 우선 적용)"],
    #     "reference": "\n".join([doc.page_content for doc in retriever.get_relevant_documents("인적공제에서 공제금액을 알려주세요.")])
    # },
    # {
    #     "question": "교육세법법 제1조가 무엇인가요?",
    #     "answer": chain.invoke("교육세법법 제1조가 무엇인가요?"),
    #     "contexts": [doc.page_content for doc in retriever.get_relevant_documents("교육세법법 제1조가 무엇인가요?")],
    #     "ground_truths": ["목적은 교육의 질적 향상을 도모하기 위하여 필요한 교육재정의 확충에 드는 재원을 확보함을 목적이다."],
    #     "reference": "\n".join([doc.page_content for doc in retriever.get_relevant_documents("교육세법법 제1조가 무엇인가요?")])
    # },
    # {
    #     "question": "개정세법 중 기업세금 감면제도의 개정 내용 요약해서 알려줘.",
    #     "answer": chain.invoke("개정세법 중 기업세금 감면제도의 개정 주요요내용 요약해서 알려줘."),
    #     "contexts": [doc.page_content for doc in retriever.get_relevant_documents("개정세법 중 기업세금 감면제도의 개정 내용 요약해서 알려줘.")],
    #     "ground_truths": ["창업중소기업 세액감면 제도 합리화, 연구･인력개발비에 대한 세액공제 확대, R&D 출연금에 대한 과세특례 인정범위 확대, 기술혁신형 중소기업 주식취득에 대한 세액공제 합리화, 성과공유 중소기업 경영성과급에 대한 세액공제 등의 적용기한 연장 및 재설계, 통합투자세액공제 제도 합리화, 지방이전지원세제 제도 정비, 소기업･소상공인 공제부금에 대한 소득공제 한도 확대, 성실사업자 등에 대한 의료비 등 세액공제 사후관리 합리화"],
    #     "reference": "\n".join([doc.page_content for doc in retriever.get_relevant_documents("개정세법 중 기업세금 감면제도의 개정 내용 요약해서 알려줘.")])
    # },
    {
        "question": "부담부 증여로 주택 취득시 장기주택저당차입금 이자상환액 공제가 가능한가가?",
        "answer": chain.invoke("부담부 증여로 주택 취득시 장기주택저당차입금 이자상환액 공제가 가능한가가?"),
        "contexts": [doc.page_content for doc in retriever.get_relevant_documents("부담부 증여로 주택 취득시 장기주택저당차입금 이자상환액 공제가 가능한가가?")],
        "ground_truths": ["증여등기일로부터 3개월 이내에 해당 주택에 저당권을 설정하고 상환기간이 15년 이상인 장기주택저당차입금을 대출받아 증여재산에 담보된 채무를 상환하는 경우 해당 채무액의 범위 내에서 이자상환액 소득공제가 가능합니다."],
        "reference": "\n".join([doc.page_content for doc in retriever.get_relevant_documents("부담부 증여로 주택 취득시 장기주택저당차입금 이자상환액 공제가 가능한가가?")])
    }

]

# Dataset 생성
dataset = Dataset.from_list(test_data)

# 평가 실행
result = evaluate(
    dataset,
    metrics=[
        faithfulness,
        answer_relevancy,
        context_precision,
        context_recall,
    ],
    llm=langchain_model,
)

# 결과 출력
print(result)

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

{'faithfulness': 0.5000, 'answer_relevancy': 0.8011, 'context_precision': 1.0000, 'context_recall': 1.0000}


In [12]:
# GPT-4o 모델을 사용하여 평가 
model_name = "gpt-4o"
model = ChatOpenAI(model=model_name)
langchain_model = LangchainLLMWrapper(model)

embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")
eval_embedding = LangchainEmbeddingsWrapper(embedding_model)
metrics = [
    LLMContextRecall(llm=langchain_model),
    LLMContextPrecisionWithReference(llm=langchain_model),
    Faithfulness(llm=langchain_model),
    AnswerRelevancy(llm=langchain_model, embeddings=eval_embedding)
]
result = evaluate(dataset=dataset, metrics=metrics)
result

Evaluating:   0%|          | 0/8 [00:00<?, ?it/s]

{'context_recall': 1.0000, 'llm_context_precision_with_reference': 0.5000, 'faithfulness': 0.0714, 'answer_relevancy': 0.2783}

In [13]:
# DataFrame 생성
df_result = pd.DataFrame(test_data)
# 최대 열 너비 설정
pd.set_option('display.max_colwidth', None)
# 최대 행 수 설정
pd.set_option('display.max_rows', None)

df_result

,question,answer,contexts,ground_truths,reference
0,개별소비세법의 목적은 무엇인가요?,"질문에 대한 답변: 개별소비세법의 목적은 사회경제적 여건 변화에 대응하여 특정 물품과 용역에 대해 부과되는 세금을 통해 재정 수입을 확보하고, 소비의 적정화 및 형평성을 도모하는 데 있습니다. 이 법은 또한 특정 물품의 소비를 억제하고 환경 보호 등의 사회적 필요를 충족시키기 위해 제정되었습니다.\n\n관련 세법 조항: 개별소비세법 제1조는 이 법의 목적을 명시하고 있습니다.\n\n추가 설명: 개별소비세는 주로 고가품, 사치품, 그리고 환경에 영향을 미치는 특정 물품에 부과되며, 이를 통해 불필요한 소비를 줄이고, 환경 보호를 촉진하는 역할을 합니다. 이러한 목적은 국가의 재정 건전성 유지에도 기여합니다. \n\n이 답변은 세법 관련 정보를 바탕으로 작성되었으며, 구체적인 상황에 따라 전문가의 추가 조언이 필요할 수 있습니다.\n\n추가 질문을 고려해 보세요:\n\n- 개별소비세가 부과되는 품목에는 어떤 것들이 있나요?\n\n- 개별소비세의 납부 절차는 어떻게 되나요?","[개별소비세법 시행규칙\n \n기획재정부 (환경에너지세제과) 044-215-4331, 4336\n기획재정부 (환경에너지세제과 - 자동차 부분) 044-215-4333, 4336, 개별소비세법 시행규칙\n \n기획재정부 (환경에너지세제과) 044-215-4331, 4336\n기획재정부 (환경에너지세제과 - 자동차 부분) 044-215-4333, 4336, 개별소비세법\n \n기획재정부 (환경에너지세제과) 044-215-4331, 4336\n기획재정부 (환경에너지세제과 - 자동차 부분) 044-215-4333, 4336]",[개별소비세법의 목적은 특정 물품과 특정 장소에 대한 소비세를 부과하여 국가 재정수입을 확보하고 소비를 조정하는 것입니다.],"개별소비세법 시행규칙\n \n기획재정부 (환경에너지세제과) 044-215-4331, 4336\n기획재정부 (환경에너지세제과 - 자동차 부분) 044-215-4333, 4336\n개별소비세법 시행규칙\n \n기획재정부 (환경에너지세제과) 044-215-4331, 4336\n기획재정부 (환경에너지세제과 - 자동차 부분) 044-215-4333, 4336\n개별소비세법\n \n기획재정부 (환경에너지세제과) 044-215-4331, 4336\n기획재정부 (환경에너지세제과 - 자동차 부분) 044-215-4333, 4336"
1,법인세법 제1조가 무엇인가요?,"법인세법 제1조는 법인세법의 목적에 대해 규정하고 있습니다. 이 조항은 법인세법의 전체적인 방향과 목적을 명확히 하고 있으며, 법인세의 과세와 관련된 기본 원칙을 제시합니다. \n\n다만, 법조문의 구체적인 내용은 질문의 범위에 따라 직접 제공할 수 없으니, 법인세법 전문을 참고하시거나 관련 법령을 확인하시기 바랍니다. \n\n이 답변은 세법 관련 정보를 바탕으로 작성되었으며, 구체적인 상황에 따라 전문가의 추가 조언이 필요할 수 있습니다.\n\n혹시 더 알고 싶으신 다른 부분이나 관련된 질문이 있으신가요?","[제1조 (목적) 이 영은 「법인세법」에서 위임된 사항과 그 시행에 필요한 사항을 규정함을 목적으로 한다., 제1조 (목적) 이 영은 「법인세법」에서 위임된 사항과 그 시행에 필요한 사항을 규정함을 목적으로 한다., 법인세법 시행규칙\n \n기획재정부 (국제조세제도과 (외국법인)) 044-215-4652\n기획재정부 (법인세제과) 044-215-4221\n 제1장 총칙]","[이 법은 법인세의 과세 요건과 절차를 규정함으로써 법인세를 공정하게 과세하고, 납세의무의 적절한 이행을 확보하며, 재정수입의 원활한 조달에 이바지함을 목적으로 한다.]",법인세법 시행규칙\n \n기획재정부 (국제조세제도과 (외국법인)) 044-215-4652\n기획재정부 (법인세제과) 044-215-4221\n 제1장 총칙\n법인세법 시행규칙\n \n기획재정부 (국제조세제도과 (외국법인)) 044-215-4652\n기획재정부 (법인세제과) 044-215-4221\n 제1장 총칙\n제1조 (목적) 이 영은 「법인세법」에서 위임된 사항과 그 시행에 필요한 사항을 규정함을 목적으로 한다.
